### This script testing Oracle EBS SSO Login functionality

In [ ]:
import time
from datetime import datetime
import selenium
from selenium.webdriver import Firefox
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.keys import Keys
from getpass import getpass

# Decorations
from PIL import Image, ImageFont, ImageDraw
from IPython.core.display import display
from IPython.core.display import Image as juPImage
from IPython.display import HTML as html_print

In [ ]:
# This Jupyter Notepad for QA testing Oracle EBS login pages R12.2.5+

test_plan = [ 
     {"url":"https://ebstimeandexpenseuat.crai.com","env_name":"CRAUAT", "u_name":"testuser"} 
    ,{"url":"https://ebstimeandexpensedev.crai.com","env_name":"CRADEV", "u_name":"testuser"}
    ,{"url":"https://www.crai.com" ,"env_name":"CRA_PUB" }
]


In [ ]:
# Enter 
u_name_ora_sysadmin ='testuser'
print('Enter password for test Oracle EBS SSO URL:')
u_name_ora_sys_pass = getpass()


In [ ]:
# Import credentials
pwd_db=[ {"e_name":"CRAUAT","username":u_name_ora_sysadmin,"pwd": u_name_ora_sys_pass} ]

In [ ]:
def get_password(db_list=None,e_name=None,username=None):
    global u_name_ora_sysadmin
    global u_name_ora_sys_pass
    global pwd_db
    d_pwd = None
    if username == u_name_ora_sysadmin:
        return True, u_name_ora_sys_pass
    for ddd in pwd_db:
        if ddd['e_name'] == e_name and ddd['username'] == username:
            d_pwd=ddd['pwd']
            break
    if d_pwd is None:
        return None
    return True, d_pwd

test = get_password(db_list=pwd_db,e_name='CRAUAT',username='SYSADMIN')


In [ ]:
# Functions
def do_open_web_page(work_url,do_snap=None):  # Login to EBS
    try:
        driver.get(work_url)
        time.sleep(3)
        page_name=driver.title
    except:
        return False,f'IERR.[url={work_url},msg=Page not available]'
    return True, page_name


def login_to_Oracle_EBS(work_url,username,passwd):  # Login to EBS
    login_url= work_url
    time.sleep(1)
    try:
        driver.get(login_url)
        time.sleep(3)
        page_name=driver.title
        print(f'Page Name:[{driver.title}]')
        if page_name == 'Sign In':
            try:
                elem = driver.find_element_by_id('userid') # For DMZ Pages
            except NoSuchElementException:
                login_text = do_check_login() ## Check if already login.
                return login_text
        elif page_name == 'Home':
            login_text = do_check_login() ## Check if already login.
            return login_text
        elif page_name == 'Login':  # Works with AppsLocalLogin.jsp
            try:
                elem = driver.find_element_by_name('usernameField') # For DMZ Pages
            except NoSuchElementException:
                return False
        else:
            print(f'Error:Oracle Login Pages Not Available: This Page:[{page_name}]')
            return False
            
        time.sleep(0.5)
        elem.send_keys(username)
        try:
            elem = driver.find_element_by_name('password')
        except:
            elem = driver.find_element_by_name('passwordField') # For DMZ Pages
        time.sleep(0.5)
        elem.send_keys(passwd)
        try:
            button = driver.find_element_by_name("submit")
        except:
            button = driver.find_element_by_class_name('OraButton')
            
        time.sleep(1)
        button.click()
        time.sleep(3)
    finally:
        try:
            ec, login_text = do_check_login(init_page_name=page_name) ## Check and validate login.
        except:
            return False
    return ec, login_text


def do_ebs_logout():  # Logout
    time.sleep(2)
    try:
        button = driver.find_element_by_xpath('//img[contains(@title, "Logout")]')
        button.click()
    except:
        print(f'Logout from UBS not found.')
        return False

def do_take_snap(file_name):
    driver.save_screenshot(file_name)

    
def crop_image(in_file,out_file):  # crop image
    x = location['x'];
    y = location['y'];
    width = location['x']+size['width'];
    height = location['y']+size['height'];
    im = Image.open('in_file')
    im = im.crop((int(x), int(y), int(width), int(height)))
    im.save(out_file)

    
def add_Test_mark(image_name,title_text,text_up=None,url_text=None,ec=None):
    url_font = ImageFont.truetype("ani.ttf", 70, encoding="unic")
    title_font = ImageFont.truetype("ani.ttf", 100, encoding="unic")
    # font = ImageFont.truetype(os.environ['LOCALAPPDATA'] + "/Microsoft/Windows/Fonts/Dance Floor.ttf", 10)
    my_image = Image.open(image_name)
    image_editable = ImageDraw.Draw(my_image)
    w=my_image.size[0]
    h=my_image.size[1]
    time_st=str(doprnTimeStamp())
    color = (237, 230, 211)  # ubuntu
    image_editable.text((w/3,90), time_st, color, font=title_font)
    color = (0, 230, 211)  # biruza
    if url_text:
        if ec is False:
            color = (237, 0, 211)  # Violet
        image_editable.text((3,h-250), url_text, color, font=url_font)
    color = (237, 230, 0)  # Yellow-Lemon
    h=my_image.size[1]
    y=h-125
    image_editable.text((15,y), title_text, color, font=title_font)
    my_image.save(image_name)
    return()

def get_file_name():
    now = datetime.now() # current date and time
    date_time = now.strftime("%m_%d_%Y_%H_%M_%S")
    return date_time

def doprnTimeStamp():
    now = datetime.now() # current date and time
    date_time = now.strftime("%d/%b/%y %H:%M")
    return date_time

def do_test_all_urls(test_plan,pwd_db,do_snap=None):
    ec=False
    ebs_login_check=True
    logout_flag=False
    total_for_check=len(test_plan)
    n_cnt=0
    for test_env in test_plan:
        n_cnt += 1
        env_url=test_env['url']
        try:
            env_name=test_env['env_name']
        except:
            do_snap=None
        try:
            login_name=test_env['u_name']
        except:
            ebs_login_check=False
            
        if ebs_login_check:
            pass_tmp = get_password(db_list=pwd_db,e_name=env_name,username=login_name)
            if pass_tmp is not None:
                pass_wd = pass_tmp[1]
            time.sleep(1)
            print(f'Checking {n_cnt}/{total_for_check} EBS: ',env_name,env_url , login_name)
            ec = login_to_Oracle_EBS(env_url,login_name,pass_wd)
            time.sleep(3)  # Wait before snapshot
            if ec is not False:
                logout_flag=True
        else:
            ec = do_open_web_page(env_url,do_snap=True)
            logout_flag=False
        if do_snap:
            file_name=f'ebs_snap_{env_name}.png'
            do_take_snap(file_name)
            title_text=f'@{env_name}'
            add_Test_mark(file_name,title_text, url_text=env_url, ec=ec)
            display(juPImage(filename=file_name, width=200))
        
        if logout_flag:
            do_ebs_logout()
        
        print(ec,f'Snap:{file_name} READY')
        time.sleep(5)
    driver.close()
    return ec

# Do_Check_Login_Page
def do_check_login(init_page_name=None):  # Works with Oracle R12.2.5+
    try:
        int_elem = driver.find_element_by_xpath('//span[contains(@class, "x2v")]').text
        usert_confirm_name = driver.find_element_by_xpath('//span[contains(@class, "x2u")]').text
    except:
        try:
            usert_confirm_name = driver.find_element_by_xpath('//div[contains(@id, "errorBox")]').text
        except:
            return False
        print('Page_Confirm:',usert_confirm_name)
        int_elem=False
    return int_elem,usert_confirm_name


# Start Testing using Firefox next step !#

In [ ]:
# All in One scan from: ________________________________________________test_plan
opts = Options()
opts.headless = False
driver = Firefox(options=opts)  # Make sure drive availabe
ec = do_test_all_urls(test_plan,pwd_db,do_snap=True)  # List of env+pwd


In [ ]:
### Demo , Testing and Development
display(html_print(f"<table><tr><td><img src='ebs_snap_CRAUAT.png'></td><td><img src='ebs_snap_CRAPROD.png'></td>\
<td><img src='ebs_snap_CRADEV.png'></td> <td><img src='ebs_snap_CRA_PUB.png'></td>   </tr></table>"))


In [ ]:
#